In [1]:
## Chapter04から使用するテキストデータをコピーする（事前にChapter04/Sentiment.ipynbを実行してください）
!cp -r ../Chapter04/sisyou_db ./

In [1]:
import time
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import spacy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from tqdm import tqdm

## モデルのロード
nlp = spacy.load("ja_ginza")

data_dir = "sisyou_db"

In [2]:
# 労災データベースからのファイルダウンロード
## 労働災害データベースからのダウンロードを下記で行っていますが、
## 本NotebookでははChapter04からコピーしているため、ダウンロード自体は行われません。
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

years = [28, 29]
months = list(range(1, 13))

for y in years:
    for m in months:
        file_url = f"https://anzeninfo.mhlw.go.jp/anzen/shisyo_xls/sisyou_db_h{y}_{m:02d}.xlsx"
        file_name = f"{data_dir}/sisyou_db_h{y}_{m:02d}.xlsx"
        if not os.path.exists(file_name):
            res = requests.get(file_url)
            print(file_name)
            with open(file_name, 'wb') as f:
                f.write(res.content)

            time.sleep(2)

In [3]:
months = list(range(1, 13))
## 労災データの読み込み
def read_rousai_db(data_dir, year_months):
    dfs = []
    for y, m in year_months:
        file_name = "{}/sisyou_db_h{}_{:02d}.xlsx".format(data_dir, y, m)
        df = pd.read_excel(file_name, skiprows=[1])
        dfs.append(df)
    df_ret = pd.concat(dfs)
    df_ret = df_ret.rename(columns={"事故の型": "事故の型_コード",
                                    "Unnamed: 20": "事故の型_名前"})
    return df_ret

In [4]:
## 訓練データ・テストデータの読み込み
year_months_train = [(28, m) for m in months]
year_months_test = [(29, 1)]
df_train = read_rousai_db(data_dir, year_months_train)
df_test = read_rousai_db(data_dir, year_months_test)

In [5]:
##  読み込んだデータフレームのベクトル化
def create_vector(df, nlp, name2label):
    docs = nlp.pipe(df["災害状況"],
                    disable=['parser', 'ner', 'morphologizer',
                             'compound_splitter', 'bunsetu_recognizer'])
    vecs = [doc.vector for doc in docs]
    X = np.array(vecs)
    y = df["事故の型_名前"].map(name2label).to_numpy()
    return X, y

In [6]:
names = df_train["事故の型_名前"].unique()
name2label = {name: i for i, name in enumerate(names)}

%time X_train, y_train = create_vector(df_train, nlp, name2label)
%time X_test, y_test = create_vector(df_test, nlp, name2label)

CPU times: user 14min 1s, sys: 32 s, total: 14min 33s
Wall time: 14min 51s
CPU times: user 1min 21s, sys: 3.16 s, total: 1min 24s
Wall time: 1min 26s


In [7]:
## 文書分類モデルの学習と評価
forest = RandomForestClassifier(n_jobs=-1)
scores = cross_val_score(forest, X_train, y_train, cv=5)
print(scores)

forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f'accuracy: {acc:.4f}')

[0.60265006 0.57790549 0.57991378 0.57320773 0.58901485]
accuracy: 0.6212


In [8]:
top10_category = df_train["事故の型_名前"].value_counts()[0:10].index.to_list()
name2label = {name: top10_category.index(name) if name in top10_category else len(top10_category) for i, name in enumerate(names)}
print(top10_category)

%time X_train, y_train = create_vector(df_train, nlp, name2label)
%time X_test, y_test = create_vector(df_test, nlp, name2label)

forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f'accuracy: {acc:.4f}')

['転倒', '墜落、転落', '動作の反動、無理な動作', 'はさまれ、巻き込まれ', '切れ、こすれ', '交通事故（道路）', '飛来、落下', '激突', '激突され', '高温・低温の物との接触']
CPU times: user 14min 2s, sys: 30.7 s, total: 14min 33s
Wall time: 14min 47s
CPU times: user 1min 21s, sys: 3.31 s, total: 1min 25s
Wall time: 1min 26s
accuracy: 0.6241
